## RNA-seq Pipeline for Detecting Exitrons (Exonic Introns)

### 0. Installs

In [1]:
import pandas as pd
import glob
import os
from tqdm import tqdm # progress tracker
import pyranges as pr # parsing gff
import numpy as np
import pysam
import itertools
import pyranges as pr # parsing gff

/gpfs/commons/home/ncui/miniconda3/envs/vscode/lib/python3.13/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### 1. Parse Regtools Data 
- Parse raw junction data from regtools output files

- Processes 1 file at a time

In [14]:
def parseJunctionFile(file_path):
    # column names for RegTools junction files
    regtools_column_names = [
        'chrom', 'start_anchor', 'end_anchor', 'name', 'score', 'strand',
        'thick_start_orig', 'thick_end_orig', 'item_rgb_orig',
        'block_count_orig', 'block_sizes_orig', 'block_starts_orig'
    ]
    
    # extract sample ID from the filename
    sample_id = os.path.basename(file_path).split('.')[0]
    
    # read the file into a pandas DataFrame
    df = pd.read_csv(
        file_path, sep='\t', header=None, names=regtools_column_names,
        dtype={'chrom': str, 'block_sizes_orig': str, 'block_starts_orig': str}
    )
        
    df['sample_id_source'] = sample_id

    # convert relevant columns to numeric types, coercing errors
    for col in ['start_anchor', 'end_anchor', 'score']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # drop rows if info is missing
    df.dropna(subset=['start_anchor', 'end_anchor', 'score', 'block_sizes_orig'], inplace=True)
    
    # ensure int types
    for col in ['start_anchor', 'end_anchor', 'score']:
        df[col] = df[col].astype(int)

    return df

### 2. Transform Junction Data
- Recalculates junction coordinates, following Regtools documentation to take into account blockSize

- Recalculates block size to represent length of junction

- Outputs junction info in BED12 format

In [15]:
def transformJunctionData(raw_df):
    
    # CHROMOSOME FILTERING
    original_row_count = len(raw_df)
    
    # allowed chromosomes
    allowed_chrom_numbers = [str(i) for i in range(1, 23)]
    allowed_sex_chroms_upper = ['X', 'Y'] 
    allowed_chromosomes = set()
    for num_chrom in allowed_chrom_numbers:
        allowed_chromosomes.add(num_chrom)
        allowed_chromosomes.add(f"chr{num_chrom}")
    for sex_chrom in allowed_sex_chroms_upper:
        allowed_chromosomes.add(sex_chrom)
        allowed_chromosomes.add(sex_chrom.lower())
        allowed_chromosomes.add(f"chr{sex_chrom}")
        allowed_chromosomes.add(f"chr{sex_chrom.lower()}")
    
    raw_df_filtered = raw_df[raw_df['chrom'].isin(allowed_chromosomes)].copy()
    filtered_row_count = len(raw_df_filtered)
    print(f"Removed {original_row_count - filtered_row_count} rows with non-standard chromosomes.")


    # JUNCTION COORD CORRECTION
    # filter rows for valid blocks
    parsed_blocks_list = raw_df_filtered['block_sizes_orig'].str.strip(',').str.split(',')
    has_sufficient_blocks = parsed_blocks_list.str.len() >= 2
    raw_df_filtered = raw_df_filtered[has_sufficient_blocks].copy()
    parsed_blocks_list = parsed_blocks_list[has_sufficient_blocks]
    
    # recalculating junction coordinates
    raw_df_filtered.loc[:, 'overhang_left'] = parsed_blocks_list.str[0].astype(int)
    raw_df_filtered.loc[:, 'overhang_right'] = parsed_blocks_list.str[1].astype(int)

    junc_start = raw_df_filtered['start_anchor'] + raw_df_filtered['overhang_left']
    junc_end = raw_df_filtered['end_anchor'] - raw_df_filtered['overhang_right']

    # filter out invalid junctions
    valid_junction = junc_start < junc_end
    raw_df_filtered = raw_df_filtered[valid_junction].copy()
    junc_start = junc_start[valid_junction]
    junc_end = junc_end[valid_junction]


    junc_length = junc_end - junc_start

    # create df
    transformed_df = pd.DataFrame()
    transformed_df['chrom'] = raw_df_filtered['chrom']
    transformed_df['chromStart'] = junc_start
    transformed_df['chromEnd'] = junc_end
    transformed_df['name'] = raw_df_filtered['name']
    transformed_df['score'] = raw_df_filtered['score']
    transformed_df['strand'] = raw_df_filtered['strand']
    transformed_df['thickStart'] = junc_start
    transformed_df['thickEnd'] = junc_end
    transformed_df['itemRgb'] = raw_df_filtered['item_rgb_orig']
    transformed_df['blockCount'] = 1
    transformed_df['blockSizes'] = junc_length.astype(str)
    transformed_df['blockStarts'] = "0"
    transformed_df['sample_id_source'] = raw_df_filtered['sample_id_source']

    print(f"Transformed {len(transformed_df)} junction records.")
    
    return transformed_df

### 3. Find Exitrons Within Junction Data
- Converts transformed junction data (transformed_df) and exon data (from gff3 file) into PyRanges objects with labels Chromosome, Start, End, Strand, and Title (a unique junction id formed by chrom:start:end:strand)

- Finds junctions that overlap with CDS regions using PyRanges method .overlap

In [16]:
# convert CDS data to PyRanges object
gff = pr.read_gff3("gencode.v48.annotation.gff3.gz")
cds = gff[gff.Feature == "CDS"]
print(f"Found {len(cds)} CDS regions.")

Found 903356 CDS regions.


In [17]:
def findExitrons(transformed_df):
    transformed_df = transformed_df[transformed_df['strand'].isin(['+', '-'])]

    # generate a unique ID for each junction (chrom:start:end:strand
    unique_id = transformed_df['chrom'].astype(str) + ':' + \
                transformed_df['chromStart'].astype(str) + ':' + \
                transformed_df['chromEnd'].astype(str) + ':' + \
                transformed_df['strand'].astype(str)

    # convert junction data to PyRanges object
    junction_pr = pr.PyRanges({'Chromosome': transformed_df['chrom'],
                    'Start': transformed_df['chromStart'],
                    'End': transformed_df['chromEnd'],
                    'Strand': transformed_df['strand'],
                    'title': unique_id,
                    'reads': transformed_df['score'],
                    'sourceID': transformed_df['sample_id_source']}) 

    # find overlapping junctions
    contained_junctions = junction_pr.overlap(cds, contained_intervals_only=True, strand_behavior='same')
    print(f"Found {len(contained_junctions)} junctions contained within CDS regions.")
            
    return contained_junctions

### 4. Compile All Exitron Info
- Iterates through each person's file, finding all exitron data then concatenating to a final matrix

- Includes sourceID (file name) and junction scores (total reads)

In [24]:
def compileExitronData(directory_path, output_filepath, file_pattern="*.bam.junc"):

    all_exitron_info = []
    file_paths = glob.glob(os.path.join(directory_path, file_pattern))
    print(f"Found {len(file_paths)} files to process.")

    # testing first 5 out of 100
    '''
    files_to_process = file_paths[:5]
    print(f"Processing the first {len(files_to_process)} files.")
    '''

    for file_path in tqdm(file_paths):
        print("Parsing new file...")
        file_name_only = os.path.basename(file_path)
        try:
            # 1.
            parsed_data = parseJunctionFile(file_path)
            # 2.
            transformed_df = transformJunctionData(parsed_data)
            # 3.
            gr_file = findExitrons(transformed_df)
            #4.
            all_exitron_info.append(gr_file)

        # skip to the next file if an error occurs
        except Exception as e:
            print(f"An error occurred while processing file {file_name_only}: {e}")
            import traceback
            traceback.print_exc()
            continue 

    # concatenate all individual data into matrix 
    final_gr = pr.concat(all_exitron_info)
    print(f"\nSuccessfully compiled exitron data from {len(all_exitron_info)} files.")
    final_gr.to_parquet(output_filepath, index=False)
    print(f"Successfully saved data to {output_filepath}")
    return final_gr

In [ ]:
compileExitronData("/gpfs/commons/groups/knowles_lab/atokolyi/als/juncs_min6bp/", "/gpfs/commons/home/ncui/project/final_exitron_data.parquet", file_pattern="*.bam.junc")
# exitron data stored in final_exitron_data.parquet

### 5. Exitron Filtering
- Returns wide dataframe of all unique exitrons with sourceIDs as columns and reads as values

In [8]:
def filterExitronData(exitron_data_filepath, min_count=30, min_peeps=10):
    long_df= pd.read_parquet(exitron_data_filepath)
    # drop duplicates created by pyranges .overlap method
    long_df.drop_duplicates(subset=['title', 'sourceID'], inplace=True)

    # create filtered wide df
    wide_df = long_df.pivot(index='title', columns='sourceID', values='reads').fillna(0)

    wide_is_gt_count = wide_df > min_count
    wide_is_gt_count_sums = wide_is_gt_count.sum(axis=1)
    keep_juncs = wide_is_gt_count_sums[wide_is_gt_count_sums >= min_peeps].index
    print(f"Found {len(keep_juncs)} junctions that passed the filter.")

    # construct final matrix
    filtered_exitron_data = wide_df.loc[keep_juncs].copy()
    filtered_exitron_data = filtered_exitron_data.where(filtered_exitron_data >= min_count, 0)

    return filtered_exitron_data

In [9]:
# filter exitron data
filtered_exitron_data = filterExitronData('final_exitron_data.parquet')
filtered_exitron_data.head()

/scratch/ipykernel_2513046/3035754114.py:7: PerformanceWarning: The following operation may generate 2610206368 cells in the resulting pandas object.
  wide_df = long_df.pivot(index='title', columns='sourceID', values='reads').fillna(0)


Found 2797 junctions that passed the filter.


sourceID,CGND-HRA-00013,CGND-HRA-00015,CGND-HRA-00017,CGND-HRA-00019,CGND-HRA-00020-2,CGND-HRA-00021,CGND-HRA-00023,CGND-HRA-00024,CGND-HRA-00025,CGND-HRA-00026,...,CGND-HRA-03135,CGND-HRA-03136,CGND-HRA-03137,CGND-HRA-03138,CGND-HRA-03139,CGND-HRA-03140,CGND-HRA-1927-1,CGND-HRA-1930-1,CGND-HRA-1941-1,CGND-HRA-1957-1
title,,,,,,,,,,,,,,,,,,,,,
chr10:119042185:119042215:-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chr10:119042185:119042245:-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,142.0,0.0,0.0,0.0,0.0
chr10:119042185:119042275:-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chr10:119042190:119042373:-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chr10:119042215:119042638:-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0


### 6. Normalization 

In [18]:
def normalizeExitronData(filtered_exitron_data, output_filepath):
    annots = pd.DataFrame({"exitron": list(filtered_exitron_data.index)})
    annots = pd.concat([annots, annots['exitron'].str.split(':', expand=True)], axis=1)
    annots.columns = ["exitron", "chr", "start", "end", "strand"]
    annots['start'] = annots['start'].astype(int)
    annots['end'] = annots['end'].astype(int)

    EXTEND = 6
    normalized_df = np.zeros(filtered_exitron_data.shape)
    ##tqdm(range(len(filtered_exitron_data.columns))):
    for i in range(len(filtered_exitron_data.columns)):
        print("Doing",i)
        source_id = filtered_exitron_data.columns[i]
        bam_filepath = f'/gpfs/commons/projects/ALS_Consortium_resource/bam_files/{source_id}.bam'

        with pysam.AlignmentFile(bam_filepath, 'rb') as bam:
            for j in range(len(filtered_exitron_data.index)):
                numerator = filtered_exitron_data.iat[j, i]

                if numerator > 0:
                    junc_chr = annots['chr'][j]
                    junc_start = annots['start'][j]
                    junc_end = annots['end'][j]

                    # calculate denominator
                    depth = bam.count_coverage(junc_chr, junc_start - EXTEND, junc_end + EXTEND, quality_threshold=0)
                    totals = [sum(values) for values in zip(*depth)]
                    
                    denom = np.median(totals[:EXTEND] + totals[-EXTEND:])
                    if denom > 0:
                        normalized_df[j, i] = numerator / denom
    np.save(output_filepath, normalized_df)
    return normalized_df

In [34]:
normalizeExitronData(filtered_exitron_data, 'normalized_data.npy')

  0%|          | 0/1876 [00:00<?, ?it/s]

### 7. Idenitfying Novel vs Annotated Exitrons

so starting an annotation matrix with our normalized junctions as rows, 
and then as columns whether the start matches the end of a known exon, 
and whether the end matches the start of a known exon. We can do this via 
set membership (i.e. junc_chr:junc_start in set of exon_chr:exon_end_pos)

Note: Each tool/data (regtools, pysam, pyranges, gtf) seems to have its own 
coordinate system, depending on whether it is 0 or 1-based, or inclusive 
or exclusive. This will impact how we match the end/start of exons to the 
start/end of junctions. I.e. we may not be able to directly match the coordinate 
without first shifting +1 or -1 bp to the start or the end (which could differ). 
You'll need to figure out the basing of the tools and the adjustments 
necessary to compare between them. The lazy way that I would do this would be to 
compute how exon matches for junction starts and ends you get when you add -1,0,+1 
to the start/end, and the one with the most matches is probably the correct shift

In [2]:
gff = pr.read_gff3("gencode.v48.annotation.gff3.gz")
exons = gff[gff.Feature == "exon"]

In [11]:
exons.head()


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,gene_id,...,exon_number,exon_id,transcript_support_level,havana_transcript,hgnc_id,havana_gene,ont,protein_id,ccdsid,artif_dupl
2,chr1,HAVANA,exon,11120,11211,.,+,.,exon:ENST00000832824.1:1,ENSG00000290825.2,...,1,ENSE00004248723.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,HAVANA,exon,12009,12227,.,+,.,exon:ENST00000832824.1:2,ENSG00000290825.2,...,2,ENSE00004248735.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,HAVANA,exon,12612,12721,.,+,.,exon:ENST00000832824.1:3,ENSG00000290825.2,...,3,ENSE00003582793.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,chr1,HAVANA,exon,13452,14413,.,+,.,exon:ENST00000832824.1:4,ENSG00000290825.2,...,4,ENSE00004248730.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,chr1,HAVANA,exon,11124,11211,.,+,.,exon:ENST00000832825.1:1,ENSG00000290825.2,...,1,ENSE00004248721.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def identifyExitrons(exitrons, exons):
    split_coords = exitrons.index.str.split(':').tolist()
    
    junctions_long = pd.DataFrame(
        split_coords,
        columns=['Chromosome', 'Start', 'End', 'Strand'],
        index=exitrons.index
    )
    junctions_long['Start'] = pd.to_numeric(junctions_long['Start'])
    junctions_long['End'] = pd.to_numeric(junctions_long['End'])

    # create sets of exon junction ends and starts
    exon_starts_set = set(exons.Chromosome.astype(str) + ':' + exons.Start.astype(str))
    exon_ends_set = set(exons.Chromosome.astype(str) + ':' + exons.End.astype(str))

    # create the exitron keys for comparison 
    exitron_start_keys = junctions_long['Chromosome'].astype(str) + ':' + (junctions_long['Start']).astype(str)
    exitron_end_keys = junctions_long['Chromosome'].astype(str) + ':' + (junctions_long['End']).astype(str)

    # create the final boolean columns and add them to the original DataFrame
    results_df = pd.DataFrame(
        {
            'start_match': exitron_start_keys.isin(exon_ends_set).values,
            'end_match': exitron_end_keys.isin(exon_starts_set).values
        },
        index=exitrons.index
    )

    return results_df

    

In [11]:
identifyExitrons(filtered_exitron_data, exons)


,start_match,end_match
title,,
chr10:119042185:119042215:-,False,False
chr10:119042185:119042245:-,False,False
chr10:119042185:119042275:-,False,False
chr10:119042190:119042373:-,False,False
chr10:119042215:119042638:-,False,False
...,...,...
chrX:76428919:76429027:+,False,False
chrX:76428955:76429063:+,False,False
chrX:77683451:77683538:-,True,True


In [17]:
identifyExitrons(filtered_exitron_data, exons)['start_match'].sum()


np.int64(4)

In [13]:
identifyExitrons(filtered_exitron_data, exons)['end_match'].sum()

np.int64(335)